# Missed Call Exploration for Guardians Hitters 2023

## Finding Change in Run Expectancy for each Base-Out-Count-Pitch

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('delta_run_exp.csv')

In [3]:
df = df[df['events']!='caught_stealing_2b']

In [4]:
df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)

In [5]:
df['on_base'] = np.where(df['on_1b'].notna(),'1','_')

In [6]:
df['on_base'] = df['on_base'] + '-' + np.where(df['on_2b'].notna(),'2','_')

In [7]:
df['on_base'] = df['on_base'] + '-' + np.where(df['on_3b'].notna(),'3','_')

In [8]:
df = df.loc[(df['description'] == 'called_strike') | (df['description'] == 'ball')]

In [9]:
df_all_calls = df.groupby(by=['player_name','description'])['pitch_type'].count().reset_index().pivot_table('pitch_type',['player_name'],'description').reset_index()

In [10]:
df['pa'] = df['game_pk'].astype(str) + '-' + df['at_bat_number'].astype(str)

In [11]:
df_grouped = df.groupby(by=['on_base','outs_when_up','count','type'])['delta_run_exp'].agg(pd.Series.mode).to_frame().reset_index()

In [12]:
df_grouped.head()

,on_base,outs_when_up,count,type,delta_run_exp
0,1-2-3,0,0-0,B,0.106
1,1-2-3,0,0-0,S,-0.126
2,1-2-3,0,0-1,B,0.110
3,1-2-3,0,0-2,B,0.135
4,1-2-3,0,1-0,B,0.195


## Finding All Missed Calls

In [13]:
def actual_strike(df):
    if df['zone'] > 10:
        return 'B'
    else:
        return 'S'

In [14]:
df['correct_type'] = df.apply(actual_strike,axis=1)

In [15]:
df_merged = df.merge(df_grouped,left_on=['on_base','outs_when_up','count','correct_type'],right_on=['on_base','outs_when_up','count','type'])

In [16]:
df_merged['missed_call_delta_run_exp'] = df_merged['delta_run_exp_x'] - df_merged['delta_run_exp_y']

In [17]:
df_merged['missed_call_delta_run_exp'].sum()

0.9909999999999997

In [18]:
df_guards = df_merged[df_merged['missed_call_delta_run_exp']!=0].groupby(by=['player_name','correct_type']).count().reset_index().iloc[0:,0:3]

In [19]:
df_guards.head()

,player_name,correct_type,pitch_type
0,"Arias, Gabriel",B,12
1,"Arias, Gabriel",S,9
2,"Bell, Josh",B,28
3,"Bell, Josh",S,26
4,"Brennan, Will",B,14


In [20]:
df_guards_pivoted = df_guards.pivot_table('pitch_type', ['player_name'], 'correct_type')

In [21]:
df_guards_pivoted.reset_index(inplace=True)

In [22]:
df_guards_pivoted.head()

correct_type,player_name,B,S
0,"Arias, Gabriel",12.0,9.0
1,"Bell, Josh",28.0,26.0
2,"Brennan, Will",14.0,15.0
3,"Calhoun, Kole",1.0,NaN
4,"Freeman, Tyler",5.0,2.0


In [23]:
df_missed_and_total = df_guards_pivoted.merge(df_all_calls,left_on='player_name',right_on='player_name')

In [24]:
df_missed_and_total.fillna(0,inplace=True)

In [25]:
df_missed_and_total.head()

,player_name,B,S,ball,called_strike
0,"Arias, Gabriel",12.0,9.0,275,102
1,"Bell, Josh",28.0,26.0,562,203
2,"Brennan, Will",14.0,15.0,317,160
3,"Calhoun, Kole",1.0,0.0,31,14
4,"Freeman, Tyler",5.0,2.0,118,63


In [26]:
df_runs = df_merged.groupby(by=['player_name','correct_type'])['missed_call_delta_run_exp'].sum().to_frame().reset_index().pivot_table('missed_call_delta_run_exp','player_name','correct_type').reset_index()

In [27]:
df_runs.rename(columns={'B':'missed_call_dre_B','S':'missed_call_dre_S'},inplace=True)

In [28]:
df_runs.head()

correct_type,player_name,missed_call_dre_B,missed_call_dre_S
0,"Arias, Gabriel",-1.996,0.792
1,"Bell, Josh",-3.735,2.997
2,"Brennan, Will",-1.434,1.513
3,"Calhoun, Kole",-0.110,0.000
4,"Freeman, Tyler",-0.476,0.078


In [29]:
df_missed_and_total_plus_runs = df_missed_and_total.merge(df_runs,left_on='player_name',right_on='player_name')

In [30]:
df_missed_and_total_plus_runs.head()

,player_name,B,S,ball,called_strike,missed_call_dre_B,missed_call_dre_S
0,"Arias, Gabriel",12.0,9.0,275,102,-1.996,0.792
1,"Bell, Josh",28.0,26.0,562,203,-3.735,2.997
2,"Brennan, Will",14.0,15.0,317,160,-1.434,1.513
3,"Calhoun, Kole",1.0,0.0,31,14,-0.110,0.000
4,"Freeman, Tyler",5.0,2.0,118,63,-0.476,0.078


In [31]:
df_guards_pivoted.to_csv('guards_calls_pivoted.csv')

In [32]:
df_missed_and_total_plus_runs.to_csv('missed_and_total_calls_plus_runs.csv')